In [35]:
import networkx as nx
import dgl
import pandas as pd
import numpy as np

跟 select action相同，只是label從action改成new_color

In [36]:
feature_df = pd.read_csv('data/change_color_df.csv')
structure_df = pd.read_csv('data/graph_structure.csv')

In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

col = feature_df.columns.tolist()
col.remove('color')
new_col = ['color_red', 'color_black', 'color_green', 'color_blue'] + col
print(new_col)



ct = ColumnTransformer([('color', OneHotEncoder(), [3])], remainder='passthrough')
feature_onehot = np.array(ct.fit_transform(feature_df))
feature_onehot_df = pd.DataFrame(feature_onehot, columns=new_col)
feature_onehot_df

['color_red', 'color_black', 'color_green', 'color_blue', 'session', 'uid', 'round', 'score', 'num_of_neighbor', 'hist_color', 'hist_neighbor', 'hist_skip', 'new_color']


,color_red,color_black,color_green,color_blue,session,uid,round,score,num_of_neighbor,hist_color,hist_neighbor,hist_skip,new_color
0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0,6.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,1.0,3.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,1.0,4.0,1.0,0.0,6.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,1.0,5.0,1.0,0.0,6.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,0.0,1.0,0.0,0.0,13.0,16.0,6.0,100.0,7.0,0.2,0.0,0.8,996.0
2149,0.0,0.0,1.0,0.0,13.0,17.0,6.0,100.0,7.0,0.2,0.2,0.6,996.0
2150,1.0,0.0,0.0,0.0,13.0,18.0,6.0,89.0,9.0,0.4,0.0,0.6,996.0
2151,0.0,0.0,0.0,1.0,13.0,19.0,6.0,100.0,7.0,0.0,0.6,0.4,996.0


In [38]:
len(feature_onehot_df[feature_onehot_df['new_color'] != 996])

468

In [39]:
def create_graph(session, df):  # df size = graph size
    size = len(df)+1
    G = nx.Graph()
    nodes = range(1, size)
    G.add_nodes_from(nodes)
    edge_list = []
    for uid in range(1, size):
        neighbor_list = df[df['uid'] == uid].iloc[0]['linked'].split(',')
        edge_list = [(int(uid), int(neighbor)) for neighbor in neighbor_list]
        G.add_edges_from(edge_list)
        G.add_edge(int(uid), int(uid))
    return G

def add_feature(graph, f_df):
    feature_col_list = ['color_red', 'color_black', 'color_green', 'color_blue', 'score', 'num_of_neighbor', 'hist_color', 'hist_neighbor', 'hist_skip']
    for n in graph.nodes():
        for f in feature_col_list:
            graph.nodes[n][f] = f_df[f_df['uid']==n][f].tolist()[0]    # 取出uid符合的user的f欄位，從series型態轉成list再取值
    return graph

In [40]:
graph_list = []
label_list = []
for session in range(1, 14):
    s_all_df = structure_df[structure_df['session'] == session]
    for round in s_all_df['round'].unique():
        
        s_df = s_all_df[s_all_df['round'] == round]
        f_df = feature_onehot_df.iloc[s_df.index]
        graph = create_graph(session, s_df)
        graph = add_feature(graph, f_df)
        graph_list.append(graph)

        labels = f_df['new_color'].tolist()
        label_list.append(labels)
len(graph_list)

106

---

In [41]:
import math
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.parameter import Parameter
from torch.nn import functional as F

In [42]:
class graphDataset(Dataset):
    def __init__(self, graph_list, label_list):
        feature_matrix_list = []
        for g in graph_list:
            feature_matrix = [list(g.nodes[j].values()) for j in range(1, g.number_of_nodes()+1)]
            feature_matrix_list.append(torch.tensor(feature_matrix, dtype=torch.float))
        
        self.feature_matrix = feature_matrix_list
        
        adj_matrix_list = []
        for g in graph_list:
            adj_matrix = nx.adjacency_matrix(g).todense()
            adj_matrix_list.append(torch.tensor(adj_matrix, dtype=torch.float))
        self.adj_matrix = adj_matrix_list

        labels = []
        for i in range(len(label_list)):
            labels.append(torch.tensor(label_list[i], dtype=torch.int64))
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.feature_matrix[idx]
        adj_matrix = self.adj_matrix[idx]
        labels = self.labels[idx]

        return features, adj_matrix, labels

In [43]:
class GraphConvolutionalLayer(nn.modules.Module):
    def __init__(self, in_feature, out_feature, bias=True):
        super(GraphConvolutionalLayer, self).__init__()
        self.in_feature = in_feature
        self.out_feature = out_feature
        self.weight = Parameter(torch.FloatTensor(in_feature, out_feature))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_feature))
        else:
            self.bias = None
    def forward(self, adj_matrix, features):
        output = torch.mm(adj_matrix, features)
        output = torch.mm(output, self.weight)
        if(self.bias is not None):
            return output + self.bias
        else:
            return output
    

class GCN_Model(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout):
        super(GCN_Model, self).__init__()
        self.gcn = GraphConvolutionalLayer(in_dim, hidden_dim)
        self.linear1 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.linear2 = torch.nn.Linear(hidden_dim, out_dim)
        self.dropout = dropout
        
    def forward(self, feature, adj_matrix):
        x = torch.nn.functional.relu(self.gcn(adj_matrix, feature))
        x = torch.nn.functional.dropout(x, self.dropout, training=self.training)
        x = torch.nn.functional.relu(self.linear1(x))
        x = torch.nn.functional.dropout(x, self.dropout, training=self.training)
        x = self.linear2(x)
        output = torch.nn.functional.softmax(x, dim=1)
        return output

In [44]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double().sum()
    return np.round(correct / len(labels), 2)

在train step只針對new_color != 996的那些node做loss function計算 -> 利用index取出來?

In [45]:
def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epoch, eval=True):
    t = time.time()
    loss_hist = []
    for _ in range(epoch):
        model.train()
        for features, adj_matrix, labels in iter(train_dataloader):
            features = features.squeeze()
            adj_matrix = adj_matrix.squeeze()
            labels = labels.squeeze()
            
            output = model(features, adj_matrix)
            # 挑出label == 996的，不參與loss計算
            excluded_index = []
            for i in range(len(labels)):
                if(labels[i] != 996):
                    excluded_index.append(i)
            if(excluded_index):
                real_labels = torch.index_select(labels, dim=0, index=torch.tensor(excluded_index))
                real_output = torch.index_select(output, dim=0, index=torch.tensor(excluded_index))
                
                loss = loss_fn(real_output, real_labels)
                loss_hist.append(float(loss))

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        
        if(eval):
            model.eval()
            total_loss = 0
            for features, adj_matrix, labels in iter(test_dataloader):
                features = features.squeeze()
                adj_matrix = adj_matrix.squeeze()
                labels = labels.squeeze()
                output = model(features, adj_matrix)
                # 挑出label == 996的，不參與loss計算
                excluded_index = []
                for i in range(len(labels)):
                    if(labels[i] != 996):
                        excluded_index.append(i)
                if(excluded_index):
                    real_labels = torch.index_select(labels, dim=0, index=torch.tensor(excluded_index))
                    # print(len(real_labels))
                    real_output = torch.index_select(output, dim=0, index=torch.tensor(excluded_index))
                    total_loss += float(loss_fn(real_output, real_labels))
            print('eval avg loss: ', total_loss/len(test_dataloader))
            print('acc: ', float(accuracy(real_output, real_labels)))
            print('='*80)
    return loss_hist

In [46]:
train_dataset = graphDataset(graph_list[:90], label_list[:90])
test_dataset = graphDataset(graph_list[90:], label_list[90:])

train_dataloader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1)

C:\Users\user\AppData\Local\Temp\ipykernel_9880\2866558056.py:12: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(g).todense()
C:\Users\user\AppData\Local\Temp\ipykernel_9880\2866558056.py:18: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  labels.append(torch.tensor(label_list[i], dtype=torch.int64))


In [47]:
model = GCN_Model(9, 9, 4, 0.2)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

l = train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epoch=20)

eval avg loss:  1.214207462966442
acc:  0.0
eval avg loss:  1.2056887075304985
acc:  0.67
eval avg loss:  1.2028753608465195
acc:  0.67
eval avg loss:  1.2003792375326157
acc:  0.67
eval avg loss:  1.1986666396260262
acc:  0.67
eval avg loss:  1.19773830473423
acc:  0.67
eval avg loss:  1.1970538794994354
acc:  0.67
eval avg loss:  1.1946364119648933
acc:  0.67
eval avg loss:  1.1930092126131058
acc:  0.67
eval avg loss:  1.19083421677351
acc:  0.67
eval avg loss:  1.1913160160183907
acc:  0.67
eval avg loss:  1.1915349513292313
acc:  0.67
eval avg loss:  1.1917064934968948
acc:  0.67
eval avg loss:  1.1914323344826698
acc:  0.67
eval avg loss:  1.190403014421463
acc:  0.67
eval avg loss:  1.1903092041611671
acc:  0.67
eval avg loss:  1.1902308240532875
acc:  0.67
eval avg loss:  1.190516658127308
acc:  0.67
eval avg loss:  1.1916406899690628
acc:  0.67
eval avg loss:  1.1925292164087296
acc:  0.67


In [48]:
f = open('color_change.txt', 'w')
for i in l:
    f.write(str(i))
    f.write('\n')

In [49]:
f.close()

In [50]:
len(l)

1680